In [30]:
import numpy as np
import pandas as pd
from numpy import pi

In [31]:

def theta_N(N):
    # составляем последовательность чисел (a, b), где a - число, b = 0 без надчеркивания, b = 1 с надчеркиванием
    N_array = []
    i = N
    while i > 0:
        N_array.append([i, 0])
        if i % 2 == 1:
            i -= 1
        else:
            i //= 2

    # добавляем надчеркивания
    flag = True
    for i in range(1, len(N_array) - 1, 1):
        if N_array[i][0] % 2 == 0 and N_array[i - 1][0] % 2 == 1 and N_array[i + 1][0] % 2 == 1:
            N_array[i][1] = 1
        if flag:
            if N_array[i - 1][0] % 2 == 1 and N_array[i][0] % 2 == 0:
                N_array[i][1] = 1
                flag = False

    # построение последовательности нечетных чисел
    theta_old = []
    theta = [1]
    for i in range(len(N_array) - 2, -1, -1):
        theta_old = [j for j in theta]
        
        # алгоритмы перехода к следующей последовательности
        # от m к 2m без надчеркивания
        if N_array[i + 1][0] * 2 == N_array[i][0] and N_array[i][1] == 0:
            m = N_array[i + 1][0]
            theta = [None for j in range(2 * m)]
            for j in range(m):
                theta[2 * j] = theta_old[j]
                theta[2 * j + 1]     = 4 * m - theta_old[j]
        # от m к 2m с надчеркиванием
        elif N_array[i + 1][0] * 2 == N_array[i][0] and N_array[i][1] == 1:
            m = N_array[i + 1][0]
            theta = [None for j in range(2 * m)]
            for j in range(m):
                theta[2 * j] = theta_old[j]
                theta[2 * j + 1]     = 4 * m - theta_old[j] + 2
        # от 2m к 2m+1 без надчеркивания
        elif N_array[i + 1][0] + 1 == N_array[i][0] and N_array[i + 1][1] == 0:
            theta.append(2 * N_array[i + 1][0] + 1)
        # от 2m к 2m+1 с надчеркиванием
        elif N_array[i + 1][0] + 1 == N_array[i][0] and N_array[i + 1][1] == 1:
            theta.append(N_array[i + 1][0] + 1)
        else:
            print("Error theta_N")
    
    return theta

# возвращает индексы для tau в правильной последовательности
def tau_id(N):
    return [(i + 1) // 2 - 1 for i in theta_N(N)]

# возвращает шаг по времени
def tau_n(N, n, mu_max, mu_min):
    # в cos() (2n + 1), т.к. в программе n = 0, 1, ..., N - 1
    return 2 / (mu_max + mu_min + (mu_max - mu_min) * np.cos(np.pi * (2 * n + 1) / (2 * N)))

# максимальное mu
def mu_max_func(K, L, h_x, h_y):
    return 4 * ((np.cos(np.pi / (2 * K)) / h_x) ** 2 + (np.cos(np.pi / (2 * L)) / h_y) ** 2)

# минимальное mu
def mu_min_func(K, L, h_x, h_y):
    return 4 * ((np.sin(np.pi / (2 * K)) / h_x) ** 2 + (np.sin(np.pi / (2 * L)) / h_y) ** 2)

# правая часть
def f(x, y, t):
    return (-1) * 25 * pi * pi * np.sin(3*pi*x) * np.sin(4*pi*y)

# начальное условие
def hi(x, y):
    return 1

# граничное условие
def phi(x, y, t):
    return 0

# аналитическое решение задачи
def u_analytical(x, y):
    return np.sin(3*pi*x) * np.sin(4*pi*y)

In [32]:
grid_mult_K = 12
grid_mult_L = 12
N = 120

In [33]:

K_0 = 6

K = grid_mult_K * (K_0 - 1) + 1
x_grid, h_x = np.linspace(0, 1, K, retstep=True)
x_grid_0 = np.linspace(0, 1, K_0)  

L_0 = 6

L = grid_mult_L * (L_0 - 1) + 1
y_grid, h_y = np.linspace(0, 1, L, retstep=True)
y_grid_0 = np.linspace(0, 1, L_0)


mu_min = mu_min_func(K, L, h_x, h_y)
mu_max = mu_max_func(K, L, h_x, h_y)
tau_array = [tau_n(N, n, mu_max, mu_min) for n in tau_id(N)]

print(f"K = {K}, L = {L}, N = {N}")

u = np.full((K, L), np.nan)
for k in range(K):
    for l in range(L):
        u[k, l] = hi(x_grid[k], y_grid[l])
u_prev = []

t = 0 
for count, tau in enumerate(tau_array):
    # print(f"count/N = {count}/{N}")

    t += tau
    u_prev = u.copy()
    u = np.full((K, L), np.nan)

    for k in range(K):
        u[k, 0]  = phi(x_grid[k], 0, t)
        u[k, -1] = phi(x_grid[k], 1, t)
    for l in range(L):
        u[0, l]  = phi(0, y_grid[l], t)
        u[-1, l] = phi(1, y_grid[l], t)
    
    for k in range(1, K - 1, 1):
        for l in range(1, L - 1, 1):
            u[k, l] = u_prev[k, l] - tau * f(x_grid[k], y_grid[l], t) + \
                (tau / h_x ** 2) * (u_prev[k + 1, l] - 2 * u_prev[k, l] + u_prev[k - 1, l]) + \
                (tau / h_y ** 2) * (u_prev[k, l + 1] - 2 * u_prev[k, l] + u_prev[k, l - 1])

u_print = u[::grid_mult_K, ::grid_mult_L]

u_an_print   = np.full((K_0, L_0), np.nan)  
u_both_print = [0 for k in range(K_0)]      
for k in range(K_0):
    u_both_print[k] = [None for l in range(L_0)]
delta_abs = np.full((K_0, L_0), np.nan)  
delta_rel = np.full((K_0, L_0), np.nan)  
norma = 0  
for k in range(K_0):
    for l in range(L_0):
        u_an_print[k, l]    = u_analytical(x_grid_0[k], y_grid_0[l])
        u_both_print[k][l]  = str(u_print[k, l]) + ' ' + str(u_an_print[k, l])
        delta_abs[k, l]     = abs(u_an_print[k, l] - u_print[k, l])
        if u_an_print[k, l] != 0:
            delta_rel[k, l] = abs(delta_abs[k, l] / u_an_print[k, l])
            norma = max(norma, delta_rel[k, l])
        else:
            delta_rel[k, l] = None

# print(f"Норма: {norma}")

K = 61, L = 61, N = 120


In [34]:
print("Значения функции. Первое число - численное решение, второе - аналитическое")
pd.DataFrame({**{"y/x": y_grid_0}, **{x_grid_0[i]:u_both_print[i] for i in range(K_0)}})

Значения функции. Первое число - численное решение, второе - аналитическое


,y/x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0,0.0,0.0 0.0,0.0 0.0,0.0 -0.0,0.0 -0.0,0.0 0.0,0.0 0.0
1,0.2,0.0 0.0,0.565518786897009 0.5590169943749476,-0.3459006277380093 -0.3454915028125263,-0.3459006277380098 -0.34549150281252605,0.5655187868970077 0.5590169943749476,0.0 2.1594879834179903e-16
2,0.4,0.0 -0.0,-0.9084897335910486 -0.9045084971874738,0.5621913875278199 0.5590169943749473,0.5621913875278204 0.5590169943749469,-0.9084897335910481 -0.9045084971874737,0.0 -3.4941249554672773e-16
3,0.6,0.0 0.0,0.9120868563502128 0.904508497187474,-0.5629868241782601 -0.5590169943749475,-0.5629868241782595 -0.559016994374947,0.9120868563502125 0.9045084971874738,0.0 3.494124955467278e-16
4,0.8,0.0 -0.0,-0.559659424809072 -0.5590169943749471,0.34949775049717297 0.34549150281252605,0.3494977504971738 0.3454915028125258,-0.5596594248090722 -0.5590169943749471,0.0 -2.1594879834179888e-16
5,1.0,0.0 -0.0,0.0 -4.65883327395637e-16,0.0 2.879317311223986e-16,0.0 2.879317311223984e-16,0.0 -4.658833273956369e-16,0.0 -1.7997117391942291e-31


In [35]:
print("Абсолютная погрешность")
pd.DataFrame({**{"y/x": y_grid_0}, **{x_grid_0[i]:delta_abs[i] for i in range(K_0)}})

Абсолютная погрешность


,y/x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.2,0.0,6.501793e-03,4.091249e-04,4.091249e-04,6.501793e-03,2.159488e-16
2,0.4,0.0,3.981236e-03,3.174393e-03,3.174393e-03,3.981236e-03,3.494125e-16
3,0.6,0.0,7.578359e-03,3.969830e-03,3.969830e-03,7.578359e-03,3.494125e-16
4,0.8,0.0,6.424304e-04,4.006248e-03,4.006248e-03,6.424304e-04,2.159488e-16
5,1.0,0.0,4.658833e-16,2.879317e-16,2.879317e-16,4.658833e-16,1.799712e-31


In [36]:
print("Относительная погрешность")
pd.DataFrame({**{"y/x": y_grid_0}, **{x_grid_0[i]:delta_rel[i] for i in range(K_0)}})

Относительная погрешность


,y/x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.2,NaN,0.011631,0.001184,0.001184,0.011631,1.0
2,0.4,NaN,0.004402,0.005679,0.005679,0.004402,1.0
3,0.6,NaN,0.008378,0.007101,0.007101,0.008378,1.0
4,0.8,NaN,0.001149,0.011596,0.011596,0.001149,1.0
5,1.0,NaN,1.000000,1.000000,1.000000,1.000000,1.0
